In [2]:
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

## Домашнее задание к лекции "Основы веб-скрапинга"
Обязательная часть

Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


In [3]:
def get_habr_articles_with_params(queries):
    habr_df  = pd.DataFrame() 
    URL = 'https://habr.com/ru/search/' 
    
    for query in queries:
        params = {
            'q': query
        }
        req = requests.get(url=URL, params=params) 
        soup = BeautifulSoup(req.text)
        
        articles = soup.find_all('article', class_='tm-articles-list__item')
        for article in articles:
            date = article.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime')
            header = article.find('h2', class_='tm-article-snippet__title').text
            link = 'https://habr.com' + article.find('a', class_='tm-article-snippet__title-link').get('href')
            
            row = {
                'date': datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.000Z'),
                'header': header,
                'link': link
            }
            habr_df = pd.concat([habr_df, pd.DataFrame([row])])

    return habr_df.reset_index(drop=True)
    
df = get_habr_articles_with_params(['python', 'анализ'])

## Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:

    кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
    в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:

<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [75]:
def get_habr_articles_with_params_count_pages(queries, count_pages=1):
    habr_df  = pd.DataFrame()
    for page in range(1, count_pages + 1):
        URL = f'https://habr.com/ru/search/page{count_pages}'
        for query in queries:
            params = {
                'q': query
            }
            req = requests.get(url=URL, params=params)
            soup = BeautifulSoup(req.text)
        
            articles = soup.find_all('article', class_='tm-articles-list__item')
            for article in articles:
                date = article.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime')
                header = article.find('h2', class_='tm-article-snippet__title').text
                link = 'https://habr.com' + article.find('a', class_='tm-article-snippet__title-link').get('href')
                count_like = article.find('div', 'tm-votes-meter tm-data-icons__item').find('span').text
            
                row = {
                    'date': datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.000Z'),
                    'header': header,
                    'link': link,
                    'count_like': count_like
                }
                habr_df = pd.concat([habr_df, pd.DataFrame([row])])
                count_pages += 1

    return habr_df.reset_index(drop=True)
    
df = get_habr_articles_with_params_count_pages(['python', 'анализ'], count_pages=1)

In [72]:
def add_description(article_df):
    for i, news in enumerate(article_df['link']):
        req = requests.get(news)
        soup = BeautifulSoup(req.text)
        html_text = soup.find_all('p')
        description = ''
        for item in html_text:
            description += item.text
        article_df.loc[i, 'description'] = description
    return article_df

In [73]:
full_df = add_description(df)